In [2]:
import mars.dataframe as md # https://pypi.org/project/pymars/
import pandas as pd
from datetime import datetime
import time
from multiprocessing import Pool
import os

In [3]:
data_path = "F:/Projects/Python/data_process/csv/tg_zd_machine-04.csv"
data_tianzheng_all = md.read_csv(data_path).to_pandas()
data_tianzheng_all.index = pd.DatetimeIndex(data_tianzheng_all["UPDATE_DATE"])
data_tianzheng_all = data_tianzheng_all.sort_index()

In [84]:
data_tianzheng_all.head()

,MACHINE_ID,ONTIME,WORKTIME,STOPTIME,QTY,PASS_QTY,REJECTS_QTY,BEAT,EFFICIENCY,STATUS,UPDATE_DATE,PDLINE_ID
UPDATE_DATE,,,,,,,,,,,,
2020-04-01 07:17:41,10,23,0,23,0,NaN,0.0,409,0.0,0,2020/4/1 7:17:41,NaN
2020-04-01 07:18:05,10,47,0,47,0,NaN,0.0,409,0.0,0,2020/4/1 7:18:05,NaN
2020-04-01 07:19:13,11,88,0,88,0,NaN,0.0,33,0.0,0,2020/4/1 7:19:13,NaN
2020-04-01 07:19:17,3,146,0,0,0,NaN,0.0,139,0.0,0,2020/4/1 7:19:17,NaN
2020-04-01 07:19:30,12,0,0,0,0,NaN,0.0,3809,0.0,0,2020/4/1 7:19:30,NaN


In [85]:
# 总数据大小
data_tianzheng_all.shape[0]

5537184

In [14]:
# 拆分装配线以及检测线
data_tianzheng_assembly = data_tianzheng_all.loc[data_tianzheng_all.PDLINE_ID.notnull()]
data_tianzheng_checking = data_tianzheng_all.loc[data_tianzheng_all.PDLINE_ID.isnull()]
print("length of assembly line data:{0}".format(data_tianzheng_assembly.shape[0]))
print("length of checking line data:{0}".format(data_tianzheng_checking.shape[0]))

length of assembly line data:3069551
length of checking line data:2467633


In [70]:
# 拆分数据输出.csv
def split_csv(data_csv):
    if(2 in data_csv["PDLINE_ID"].values.tolist()):
        for i in range(1,13):
            output = data_csv[data_csv["MACHINE_ID"].isin([i])]
            output.to_csv('F:/Projects/Python/data_process/csv/splited_data/data_tianzheng_assembly_{0}.csv'.format(i),
                          index=None)
    else:
        for i in range(1,13):
            output = data_csv[data_csv["MACHINE_ID"].isin([i])]
            output.to_csv('F:/Projects/Python/data_process/csv/splited_data/data_tianzheng_checking_{0}.csv'.format(i),
                          index=None)          

In [71]:
# 输出.csv别手贱运行！
# split_csv(data_tianzheng_assembly)
# split_csv(data_tianzheng_checking)

In [194]:
# 输入一个拆分后的数据
data_tianzheng_assembly = md.read_csv('F:/Projects/Python/data_process/csv/splited_data/data_tianzheng_assembly_2.csv').to_pandas()
data_tianzheng_assembly.index = pd.DatetimeIndex(data_tianzheng_assembly["UPDATE_DATE"])
print(data_tianzheng_assembly.shape[0])

304524


In [10]:
# 丢弃工作时间错误的数据
def process_error_data_01(data):   
    for index, row in data.iterrows():
        if row['ONTIME'] != row['WORKTIME'] + row['STOPTIME']:
#             print(index, row['MACHINE_ID'] ,row['PDLINE_ID'])
            data.drop(index = index, axis=0, inplace=True)

In [11]:
# 单核处理
start = time.time()
process_error_data_01(data_tianzheng_assembly)
end = time.time()
print('Processing time %0.2f seconds.' % (end - start))

Processing time 214.02 seconds.


In [175]:
# 得到文件中的日期
# https://www.cnblogs.com/nxf-rabbit75/p/10662025.html
def get_date_list(data):
    process_data = data.copy()
    process_data = process_data.resample('D').mean()
    process_data = process_data.dropna(axis=0,how="all")
    process_date_list = process_data.index.strftime('%Y-%m-%d').tolist() # https://www.runoob.com/python/att-time-strftime.html

    #  用完删除，释放内存
    del process_data
    import gc
    gc.collect()
    
    return process_date_list

In [192]:
# 判断后一天开始时的QTY是否清零，即是否小于前一天结束时的QTY
def process_error_data_02(data, date_list):
    error = False
    for i in range(len(date_list)-1):
        QTY_01 = int(data.loc[date_list[i]][-1:]['QTY'])
        QTY_02 = int(data.loc[date_list[i+1]][:1]['QTY'])
        if QTY_02 >= QTY_01:
            print("QTY of the day before:\n",data.loc[date_list[i]][-1:][['QTY']],'\n')
            print("QTY of the day after:\n",data.loc[date_list[i+1]][:1][['QTY']],'\n')
            error = True
    if error == False:
        print("QTY data are all correct!")

In [ ]:
process_error_data_02(data_tianzheng_assembly, get_date_list(data_tianzheng_assembly))

QTY of the day before:
                       QTY
UPDATE_DATE              
2020-04-01 12:52:18  4396 

QTY of the day after:
                       QTY
UPDATE_DATE              
2020-04-16 13:21:04  4977 

